# Processing data for PVM Analysis
The goal of this notebook is to process the sales data so that we can build the PVM analysis for "Desafinados Karaoke Lounge" - a fake karaoke business that operates in Japan). We will first create several supporting columns and then slice and dice the table to build the version of the data that we need for Tableau.

In Tableau, we will build waterfall charts. These charts will show how individual sales drivers have contributed to changes in monthly sales over a period of 1 year. Here's what I mean:

- Sales in April 2020 = X
  -  (Driver 1) Change in sales attributable to change in unit price = p
  -  (Driver 2) Change in sales attributable to change in average visit = v
  -  (Driver 3) Change in sales attributable to change member count = m 
- Sales in April 2021 = X + p + v + m

Ideally, the karaoke business wants to grow its active customer base (m), grow the frequency that customers come to their venue (v), and grow the money spent by selling drinks, snacks, and other optional features (p). 

In [99]:
import pandas as pd

# Load the dummy data
df = pd.read_csv("basic_salesdata.csv")

In [101]:
# dummy data looks like this
df.head()

,date,store,sales,totalvisit,membercnt
0,2020-04-30,Shinjuku Central,163602,227,178
1,2020-05-31,Shinjuku Central,142562,281,146
2,2020-06-30,Shinjuku Central,154344,325,161
3,2020-07-31,Shinjuku Central,166968,237,143
4,2020-08-31,Shinjuku Central,189017,286,162


In [104]:
# "Desafindos" has 10 karaoke lounges in Japan
df["store"].unique()

array(['Shinjuku Central', 'Harajuku Boulevard', 'Shinagawa Square',
       'Tokyo Bay', 'Yokohama Port', 'Osaka Metro', 'Osaka Sunset Plaza',
       'Nagoya Tower', 'Hakata Outlet', 'Okinawa Seaside'], dtype=object)

In [106]:
# This is sales data for 24 months of business. Includes 2 fiscal years (FY21: 2020-4 to 2021-3 and FY22: 2021-4 to 2022-3).
df["date"].unique()

array(['2020-04-30', '2020-05-31', '2020-06-30', '2020-07-31',
       '2020-08-31', '2020-09-30', '2020-10-31', '2020-11-30',
       '2020-12-31', '2021-01-31', '2021-02-28', '2021-03-31',
       '2021-04-30', '2021-05-31', '2021-06-30', '2021-07-31',
       '2021-08-31', '2021-09-30', '2021-10-31', '2021-11-30',
       '2021-12-31', '2022-01-31', '2022-02-28', '2022-03-31'],
      dtype=object)

In [87]:
# Build necessary columns
df["month"] = df["date"].apply(lambda x: x.split("-")[1])
df["yearmonth"] = df["date"].apply(lambda x: (x.split("-")[0])) + df["date"].apply(lambda x: x.split("-")[1])
df["yearmonth"] = df["yearmonth"].astype("int")
df["avgprice"] = df["sales"] / df["totalvisit"]
df["avgvisit"] = df["totalvisit"] / df["membercnt"]
df["FY"] = df["yearmonth"].apply(lambda x: "FY21" if x <= 202103 else "FY22")
df.head()

,date,store,sales,totalvisit,membercnt,month,yearmonth,avgprice,avgvisit,FY
0,2020-04-30,Shinjuku Central,163602,227,178,04,202004,720.713656,1.275281,FY21
1,2020-05-31,Shinjuku Central,142562,281,146,05,202005,507.338078,1.924658,FY21
2,2020-06-30,Shinjuku Central,154344,325,161,06,202006,474.904615,2.018634,FY21
3,2020-07-31,Shinjuku Central,166968,237,143,07,202007,704.506329,1.657343,FY21
4,2020-08-31,Shinjuku Central,189017,286,162,08,202008,660.898601,1.765432,FY21


Let's start the data processing work

In [88]:
# Divide the table into two tables: one for each fiscal year
df_FY21 = df[df["FY"] == "FY21"]
df_FY22 = df[df["FY"] == "FY22"]

# Merge fiscal year tables so that all is aligned for easy calculation
df_prepcalc = df_FY21.merge(df_FY22, left_on=["store", "month"], right_on=["store", "month"])
df_prepcalc.head()

,date_x,store,sales_x,totalvisit_x,membercnt_x,month,yearmonth_x,avgprice_x,avgvisit_x,FY_x,date_y,sales_y,totalvisit_y,membercnt_y,yearmonth_y,avgprice_y,avgvisit_y,FY_y
0,2020-04-30,Shinjuku Central,163602,227,178,04,202004,720.713656,1.275281,FY21,2021-04-30,183799,282,145,202104,651.769504,1.944828,FY22
1,2020-05-31,Shinjuku Central,142562,281,146,05,202005,507.338078,1.924658,FY21,2021-05-31,149295,323,137,202105,462.213622,2.357664,FY22
2,2020-06-30,Shinjuku Central,154344,325,161,06,202006,474.904615,2.018634,FY21,2021-06-30,146434,294,184,202106,498.074830,1.597826,FY22
3,2020-07-31,Shinjuku Central,166968,237,143,07,202007,704.506329,1.657343,FY21,2021-07-31,149463,266,127,202107,561.890977,2.094488,FY22
4,2020-08-31,Shinjuku Central,189017,286,162,08,202008,660.898601,1.765432,FY21,2021-08-31,201304,292,175,202108,689.397260,1.668571,FY22


In [89]:
# Reordering the columns
df_prepcalc = df_prepcalc[['store', 'month', 'date_x', 'sales_x', 'totalvisit_x', 'membercnt_x',
       'yearmonth_x', 'avgprice_x', 'avgvisit_x', 'FY_x', 'date_y', 'sales_y',
       'totalvisit_y', 'membercnt_y', 'yearmonth_y', 'avgprice_y',
       'avgvisit_y', 'FY_y']]

In [90]:
# Calculate the sales effect for each driver
df_prepcalc["member_effect"] = df_prepcalc.apply(lambda x: (x["membercnt_y"] - x["membercnt_x"]) * x["avgvisit_x"] * x["avgprice_x"] , axis=1)
df_prepcalc["avgvisit_effect"] = df_prepcalc.apply(lambda x: (x["avgvisit_y"] - x["avgvisit_x"]) * x["membercnt_y"] * x["avgprice_x"] , axis=1)
df_prepcalc["avgprice_effect"] = df_prepcalc.apply(lambda x: (x["avgprice_y"] - x["avgprice_x"]) * x["membercnt_y"] * x["avgvisit_y"] , axis=1)


In [92]:
# Rename columns
newcols = [x.replace("_x", "_1").replace("_y", "_2") for x in df_prepcalc.columns]
newcols_map = dict(zip(df_prepcalc.columns, newcols))
df_prepcalc.rename(columns=newcols_map, inplace=True)

# Keep columns for waterfall chart
df_bridge = df_prepcalc[['store', 'month', 'date_1', 'date_2', 'FY_1', 'FY_2','sales_1', "member_effect", "avgvisit_effect", "avgprice_effect", 'sales_2']]
df_bridge_melt = df_bridge.melt(id_vars=["store", "month", "date_1", "date_2" ,"FY_1", "FY_2"])

# Pick a single store / month and check calculations:
df_bridge_melt[(df_bridge_melt["store"] == "Okinawa Seaside") & (df_bridge_melt["month"] == "04")] 


,store,month,date_1,date_2,FY_1,FY_2,variable,value
108,Okinawa Seaside,04,2020-04-30,2021-04-30,FY21,FY22,sales_1,61878.000000
228,Okinawa Seaside,04,2020-04-30,2021-04-30,FY21,FY22,member_effect,16524.238636
348,Okinawa Seaside,04,2020-04-30,2021-04-30,FY21,FY22,avgvisit_effect,-30984.858202
468,Okinawa Seaside,04,2020-04-30,2021-04-30,FY21,FY22,avgprice_effect,13996.619565
588,Okinawa Seaside,04,2020-04-30,2021-04-30,FY21,FY22,sales_2,61414.000000


# Build Supporting Table

Necessary data to build the waterfall chart is ready. To support our Tableau dashboard, I will build a KPI table that will show numerically how each driver changed from one period to the other.

In [93]:
# get necessary columns
df_kpi = df_prepcalc[["store", "month", "date_1", "date_2" ,"FY_1", "FY_2", "sales_1", "avgprice_1", "avgvisit_1", "membercnt_1", "sales_2", "avgprice_2", "avgvisit_2", "membercnt_2"]]
df_kpi_melt = df_kpi.melt(id_vars=["store", "month", "date_1", "date_2" ,"FY_1", "FY_2"])

In [94]:
# break table into two: one has data from the period 1 and other from period 2
df_kpi_melt2 = df_kpi_melt[df_kpi_melt["variable"].isin(["sales_2", "avgprice_2", "avgvisit_2", "membercnt_2"])]
df_kpi_melt1 = df_kpi_melt[df_kpi_melt["variable"].isin(["sales_1", "avgprice_1", "avgvisit_1", "membercnt_1"])]

# melt both data
df_kpi_melt2["variable"] = df_kpi_melt2.apply(lambda x : x["variable"].replace("_2",""), axis=1)
df_kpi_melt1["variable"] = df_kpi_melt1.apply(lambda x : x["variable"].replace("_1",""), axis=1)

# merge and align
df_kpi_melt_tbl = df_kpi_melt1.merge(df_kpi_melt2,
    left_on=["store","month","date_1","date_2","FY_1","FY_2","variable"],
    right_on=["store","month","date_1","date_2","FY_1","FY_2","variable"])



C:\Users\ochib\AppData\Local\Temp\ipykernel_19232\2572064843.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_kpi_melt2["variable"] = df_kpi_melt2.apply(lambda x : x["variable"].replace("_2",""), axis=1)
C:\Users\ochib\AppData\Local\Temp\ipykernel_19232\2572064843.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_kpi_melt1["variable"] = df_kpi_melt1.apply(lambda x : x["variable"].replace("_1",""), axis=1)


In [95]:
# rename value column
df_kpi_melt_tbl.rename(columns={"value_x":"value_1","value_y":"value_2"}, inplace=True)

# check data for a single store / month
df_kpi_melt_tbl[(df_kpi_melt_tbl["store"] == "Shinjuku Central") & (df_kpi_melt_tbl["month"] == "04")]

,store,month,date_1,date_2,FY_1,FY_2,variable,value_1,value_2
0,Shinjuku Central,04,2020-04-30,2021-04-30,FY21,FY22,sales,163602.000000,183799.000000
120,Shinjuku Central,04,2020-04-30,2021-04-30,FY21,FY22,avgprice,720.713656,651.769504
240,Shinjuku Central,04,2020-04-30,2021-04-30,FY21,FY22,avgvisit,1.275281,1.944828
360,Shinjuku Central,04,2020-04-30,2021-04-30,FY21,FY22,membercnt,178.000000,145.000000


In [81]:
# Export into csv
df_kpi_melt_tbl.to_csv("kpitbl.csv", index=False)
df_bridge_melt.to_csv("bridge.csv", index=False)

Next step is to build the waterfall charts in Tableau. Find the dashboard [here](https://public.tableau.com/app/profile/bochi/viz/PVMAnalysis/PVMAnalysis) (hosted in Tableau Public).